# Experimento: Transcripción con Whisper 'large' en GPU (Colab)

Este cuaderno te guía paso a paso para ejecutar el modelo `large` de OpenAI Whisper en una GPU gratuita de Google Colab. El objetivo es replicar un experimento básico para:

1.  **Medir el tiempo de carga** del modelo `large`.
2.  **Medir el tiempo de transcripción** de un archivo de audio de ejemplo.
3.  **Observar el uso de recursos** (VRAM de la GPU y RAM del sistema).
4.  **Evaluar la calidad** de la transcripción resultante.

Está basado en el proyecto [Audio-WhatsApp-a-texto](https://github.com/ilahuerta-IA/Audio-WhatsApp-a-texto), utilizando algunos de sus módulos auxiliares.

**¡Importante!** Asegúrate de haber configurado el entorno de ejecución para usar una **GPU** (`Entorno de ejecución` -> `Cambiar tipo de entorno de ejecución` -> `GPU`).

In [ ]:
# -*- coding: utf-8 -*-
# Celda 2: Verificar GPU y Clonar Repositorio Base

import torch
import os

# Verificar si la GPU está disponible
if torch.cuda.is_available():
    print(f"GPU detectada: {torch.cuda.get_device_name(0)}")
    DEVICE = "cuda"
else:
    print("ADVERTENCIA: No se detectó GPU. Selecciona una GPU en 'Entorno de ejecución'.")
    print("El modelo 'large' NO funcionará razonablemente en CPU.")
    DEVICE = "cpu" # Continuar en CPU es solo para fines demostrativos del código

# Clonar el repositorio original que contiene los módulos auxiliares
# (solo necesitamos audio_handler.py y config.py de él)
repo_url = "https://github.com/ilahuerta-IA/Audio-WhatsApp-a-texto.git"
repo_name = "Audio-WhatsApp-a-texto"
if not os.path.exists(repo_name):
    print(f"Clonando repositorio desde {repo_url}...")
    !git clone {repo_url}
else:
    print(f"Directorio '{repo_name}' ya existe.")

# Cambiar al directorio del repositorio clonado
try:
    os.chdir(repo_name)
    print(f"Directorio de trabajo cambiado a: {os.getcwd()}")
except FileNotFoundError:
    print(f"ERROR: No se pudo cambiar al directorio '{repo_name}'. ¿Falló la clonación?")

### Paso 1: Instalar Dependencias

Necesitamos instalar algunas herramientas:
*   `ffmpeg`: Esencial para que `pydub` pueda leer y convertir diferentes formatos de audio.
*   `openai-whisper`: La librería principal para la transcripción.
*   `pydub`: Para manejar la conversión de audio a WAV, formato preferido por Whisper.

In [ ]:
# Celda 4: Instalar ffmpeg, whisper y pydub

print("Instalando ffmpeg...")
!apt-get update -qq && apt-get install -y ffmpeg -qq
print("Instalando openai-whisper y pydub...")
!pip install -U openai-whisper pydub -q

print("\nDependencias instaladas.")

### Paso 2: Sube tu Archivo de Audio

Ahora es el momento de **subir tu propio archivo de audio** a Colab.

1.  Mira el panel de la izquierda y busca el icono de **Carpeta** (`Archivos`).
2.  Haz clic en el icono **Subir** (📄↑).
3.  Selecciona el archivo de audio desde tu ordenador (`.mp3`, `.wav`, `.m4a`, `.ogg`, etc.).
4.  Espera a que termine la subida. El archivo aparecerá en el panel de archivos, normalmente en la carpeta `/content/` (o dentro de la carpeta `Audio-WhatsApp-a-texto` si la tenías seleccionada al subir).
5.  **¡MUY IMPORTANTE!** Haz clic derecho sobre el archivo subido y selecciona **`Copiar ruta`**. Necesitarás esta ruta en la siguiente celda.

In [ ]:
# Celda 6: Configurar Experimento y Cargar Módulos

import pathlib
import time
import sys
import os

# --- ¡¡MODIFICA ESTA LÍNEA CON LA RUTA DE TU ARCHIVO!! ---
# Pega aquí la ruta que copiaste en el paso anterior.
# Ejemplo: "/content/mi_audio_subido.mp3" o "/content/Audio-WhatsApp-a-texto/mi_audio.wav"
RUTA_AUDIO_ORIGINAL_STR = "/content/mi_audio.wav" # <-- ¡¡CAMBIA ESTO!!
# ---------------------------------------------------------

MODELO_A_PROBAR = "large"
# La Tesla T4 soporta FP16, puede ser más rápido y usar menos VRAM.
# Ponlo en False si quieres comparar o si usas una GPU más antigua (como K80).
USE_FP16 = False if DEVICE == "cpu" else True # Usar True por defecto en GPU

# --- Añadir directorio actual (donde están los .py clonados) a sys.path ---
repo_directorio = os.getcwd() # Ya estamos dentro del dir clonado
if repo_directorio not in sys.path:
    sys.path.insert(0, repo_directorio)
    print(f"Directorio '{repo_directorio}' añadido a sys.path.")

# --- Importar Módulos Auxiliares ---
try:
    import audio_handler
    import config
    print("Módulos auxiliares importados correctamente.")
except ImportError as e:
    print(f"ERROR importando módulos: {e}. Asegúrate de estar en el directorio correcto.")

# --- Validar Ruta y Crear Variables ---
try:
    ruta_audio_original = pathlib.Path(RUTA_AUDIO_ORIGINAL_STR)
    if not ruta_audio_original.is_file():
         raise FileNotFoundError(f"¡Archivo no encontrado en la ruta especificada! Verifica RUTA_AUDIO_ORIGINAL_STR.")
    else:
        print(f"Archivo de audio a procesar: {ruta_audio_original}")
except Exception as e:
    print(f"Error con la ruta del archivo: {e}")
    ruta_audio_original = None

# Variables para resultados
modelo_cargado = None
ruta_audio_wav = None
tiempo_carga_modelo = 0
tiempo_transcripcion = 0
resultado_transcripcion = None

print(f"\nConfiguración lista. Modelo a usar: '{MODELO_A_PROBAR}', Dispositivo: {DEVICE}, FP16: {USE_FP16}")

### Paso 3: Convertir Audio a WAV

Whisper funciona mejor con archivos WAV. Usaremos el módulo `audio_handler` (del repositorio base) para convertir tu archivo subido a un formato WAV temporal estándar.

In [ ]:
# Celda 8: Convertir a WAV

if ruta_audio_original:
    print(f"Convirtiendo {ruta_audio_original.name} a WAV temporal...")
    start_time_convert = time.time()
    ruta_audio_wav = audio_handler.convert_to_wav_if_needed(ruta_audio_original)
    end_time_convert = time.time()

    if ruta_audio_wav and ruta_audio_wav.exists():
        print(f"Audio convertido a: {ruta_audio_wav} en {end_time_convert - start_time_convert:.2f} segundos.")
    else:
        print("Error: Falló la conversión a WAV.")
else:
    print("Error: No hay ruta de audio original válida para convertir.")

### Paso 4: Cargar Modelo Whisper 'large'

Ahora cargaremos el modelo `large` en la memoria de la GPU (o CPU si no hay GPU). Mediremos cuánto tiempo tarda este proceso.

**¡Atención!** Durante la ejecución de esta celda:
*   **Observa los indicadores `RAM` y `GPU RAM`** en la esquina superior derecha de Colab. Anota el uso máximo que veas.
*   Whisper descargará el modelo si es la primera vez que se usa en esta sesión (puede tardar un poco).

In [ ]:
# Celda 10: Cargar Modelo y Medir Tiempo/Recursos

import whisper # Asegurarnos de importar la librería principal

if DEVICE != "cpu": # Solo cargar si hay GPU o si aceptamos la lentitud de CPU
    print(f"\n--- Cargando modelo Whisper '{MODELO_A_PROBAR}' en {DEVICE} ---")
    start_time_load = time.time()
    try:
        # Carga directa usando la librería whisper
        modelo_cargado = whisper.load_model(MODELO_A_PROBAR, device=DEVICE)
        end_time_load = time.time()
        tiempo_carga_modelo = end_time_load - start_time_load
        print(f"Modelo '{MODELO_A_PROBAR}' cargado en {tiempo_carga_modelo:.2f} segundos.")

        print("\n>> ¡REVISA AHORA los indicadores RAM y GPU RAM de Colab para ver el uso MÁXIMO durante la carga! <<")
        print("Salida de nvidia-smi (uso de VRAM actual):")
        !nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader,nounits

    except Exception as e:
        print(f"Error crítico al cargar el modelo: {e}")
        modelo_cargado = None
else:
    print("Proceso detenido porque no se detectó GPU o se eligió CPU (no recomendado para 'large').")

### Paso 5: Realizar la Transcripción

Con el modelo cargado y el audio en formato WAV, realizamos la transcripción. Mediremos el tiempo y volveremos a observar el uso de recursos.

**¡Atención!** Durante la ejecución de esta celda:
*   **Observa de nuevo los indicadores `RAM` y `GPU RAM`**. El pico de uso de VRAM suele ocurrir *durante* la transcripción.
*   La opción `fp16` (definida en la Celda 6) controla si se usa precisión media (más rápido, menos VRAM) o completa (predeterminada).

In [ ]:
# Celda 12: Transcribir y Medir Tiempo/Recursos

if modelo_cargado and ruta_audio_wav:
    print(f"\n--- Transcribiendo {ruta_audio_wav.name} con '{MODELO_A_PROBAR}' (fp16={USE_FP16}) ---")
    start_time_transcribe = time.time()
    try:
        # Asegúrate de pasar la ruta como string
        audio_path_str = str(ruta_audio_wav)

        # ¡Llamada a la transcripción!
        # verbose=None o False para menos salida en consola, True para detalles
        resultado_transcripcion = modelo_cargado.transcribe(
            audio_path_str,
            language=config.TARGET_LANGUAGE, # Definido en config.py ('es')
            initial_prompt=config.WHISPER_INITIAL_PROMPT, # Definido en config.py
            fp16=USE_FP16,
            verbose=None
        )
        end_time_transcribe = time.time()
        tiempo_transcripcion = end_time_transcribe - start_time_transcribe
        print(f"Transcripción completada en {tiempo_transcripcion:.2f} segundos.")

        print("\n>> ¡REVISA AHORA los indicadores RAM y GPU RAM de Colab para ver el uso MÁXIMO durante la transcripción! <<")
        print("Salida de nvidia-smi (uso de VRAM actual post-transcripción):")
        !nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader,nounits

    except Exception as e:
        print(f"Error crítico durante la transcripción: {e}")
        resultado_transcripcion = None
else:
    print("No se puede transcribir: el modelo no se cargó o el archivo WAV no está listo.")

### Paso 6: Ver Resultados

Mostramos un resumen de los tiempos medidos y el texto transcrito obtenido.

In [ ]:
# Celda 14: Mostrar Resumen y Texto Transcrito

print("\n--- Resumen de la Ejecución ---")
print(f"Modelo: {MODELO_A_PROBAR}")
print(f"Dispositivo: {DEVICE} (FP16={USE_FP16})")
if ruta_audio_original:
  print(f"Archivo Original: {ruta_audio_original.name}")
else:
  print("Archivo Original: (Error en ruta)")
print(f"Tiempo de Carga del Modelo: {tiempo_carga_modelo:.2f} s")
print(f"Tiempo de Transcripción: {tiempo_transcripcion:.2f} s")

if resultado_transcripcion:
    print("\n--- Texto Transcrito ---")
    # Accedemos al texto dentro del diccionario resultado
    texto_final = resultado_transcripcion.get("text", "Error: No se encontró texto en el resultado.")
    print(texto_final.strip()) # .strip() para quitar espacios iniciales/finales
    print("-" * 26) # Separador

    # (Opcional) Descomenta para ver los segmentos individuales con tiempos
    # print("\n--- Segmentos Detallados ---")
    # segments = resultado_transcripcion.get("segments", [])
    # if segments:
    #     for segment in segments:
    #         start = segment.get('start', 0)
    #         end = segment.get('end', 0)
    #         text = segment.get('text', '')
    #         print(f"[{start:>7.2f}s -> {end:>7.2f}s] {text.strip()}")
    # else:
    #     print("No se encontraron segmentos detallados en el resultado.")

else:
    print("\nNo se obtuvo resultado de la transcripción debido a errores previos.")

### Paso 7: Limpieza

Finalmente, eliminamos el archivo WAV temporal que se creó durante la conversión.

In [ ]:
# Celda 16: Limpiar Archivo Temporal

# Usar la función de limpieza de audio_handler
# para eliminar el archivo _temp_playback.wav si existe
try:
    if 'audio_handler' in sys.modules:
        print("\nLimpiando archivo WAV temporal...")
        audio_handler.cleanup_temp_wav()
        print("Limpieza completada.")
    else:
        print("Módulo audio_handler no cargado, no se puede limpiar automáticamente.")
except Exception as e:
    print(f"Error durante la limpieza del archivo temporal: {e}")

### Conclusión del Experimento

¡Has ejecutado Whisper `large` en una GPU!

*   Compara los **tiempos de carga y transcripción** con lo que experimentarías en CPU.
*   Analiza el **uso de VRAM y RAM** que observaste. ¿Cabe en GPUs comunes?
*   Revisa la **calidad del texto transcrito**. ¿Cumple tus expectativas?

Puedes volver a ejecutar las celdas (especialmente desde la Celda 6) cambiando el archivo de audio, el modelo (`medium`, `small`...) o el parámetro `USE_FP16` para comparar resultados.